

------


## 🥑 About The Project

**Introduction**

What is being tried to accomplish from this project?
Educational exploration of machine learning models applied to a dataset of female breast cancer observations.

Project Description/Abstract
This Work is dedicated to my aunt Larisa , that i hope gets well.
The goal of this project is to pull a variety of cancer dtasets available online into single analysis that will contain both visualisation and  perdiction aspects.
Wisconsin Breast Cancer Dataset is best suited for the machine laerning exploration as it will require dimentionality reduction , hence making it more complex.
Observations in this dataset are measurements of images of Female breast cancer tumor cells.

Dataset being used and descriptions of the dataset:
* Attribute Information:
*  ID number
*  Diagnosis (M = malignant, B = benign)
* Ten real-valued features are computed for each cell nucleus:
* a) radius (mean of distances from center to points on the perimeter)
* b) texture (standard deviation of gray-scale values)
* c) perimeter
* d) area
* e) smoothness (local variation in radius lengths)
* f) compactness (perimeter^2 / area - 1.0)
* g) concavity (severity of concave portions of the contour)
* h) concave points (number of concave portions of the contour)
* i) symmetry
* j) fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three
largest values) of these features were computed for each image,
resulting in 30 features. For instance, field 3 is Mean Radius, field
13 is Radius SE, field 23 is Worst Radius.

All feature values are recoded with four significant digits.

**Missing attribute values: none**

**Class distribution: 357 benign, 212 malignant**
This Dataset can be found on UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29
## Project methodology 

- [🥝  Import The Data](#1.0 )

- [🍍 Explore The Data](#1.2)

- [🍄  PreProcess The Data](#1.3)

- [🍒 Model & Validate The Data](#2.0)
 - **[Principal Component Analysis](#PCA)
 - **[Multinominal Logistic Regression](#GLM)
 - **[Support Vector Machines](#SVM)
 - **[K Nearest Neighbour](#KNN)
 - **[C 5.0](#C5)
 - **[Cost Sensitive C 5.0 ](#csC5)
     
- [🥒 Conclude The Data](#3.0)


 #1.0
 ## Import  The Data  🥝
 

In [ ]:
options(warn=-1)
warning=FALSE
library("ggplot2")
library("GGally")
library("caret")
library("tidyverse")
library("viridisLite")
library("funModeling")

library("gridExtra")


data.cancer1 = read.csv("../input/cancer69/data.csv")





#1.2

## 🍍   Explore The Data

In [ ]:
str(data.cancer1)

In [ ]:
data.cancer1$X <- NULL
data.cancer1$id <- NULL

In [ ]:
options(repr.plot.width=13, repr.plot.height = 10)
plot_num(log(data.cancer1[,-1]))

In [ ]:
library(repr) ;





cancer=data.cancer1[c(-1,-25)]
ggcorr(cancer,label=T)



In [ ]:
library(dplyr)
freq(data.cancer1$diagnosis)

#1.3
##  🍄  PreProcess The Data

In [ ]:
set.seed(111)
 index= sample(2, nrow(data.cancer1),
              replace = TRUE,
              prob = c(0.8, 0.2))
train = data.cancer1[index==1,]
test = data.cancer1[index==2,]

#PCA-Principal Component Analysis 
 
 PCA is a linear dimentionality reduction function that combines miltiple variables into one that is projected  on a single dimention.
 It is used to help the user choose which data combination perfoms better explaining the variance of given dataset.
 In our case PC1 , the first component, explains 44% of variability in the given dataset using 31 predictors.
 Thses componets are later used as predictors for the machine learning algorithms.
 
 Depending on users preference ,one can choose as much as all componets, or as little as one .
 We shall choose 2 to begin with , PC1 amd PC2.

In [ ]:
pca <- prcomp(train[,-1],
             center = TRUE,
             scale. = TRUE)
           
summary(pca)

Here is the Variable Contribution Chart which represents which particular column is more dominant in explaining the variance.  

In [ ]:
library("gridExtra")
library("FactoMineR")
library("factoextra")
options(repr.plot.width=16, repr.plot.height = 8)
p1 <- fviz_contrib(pca, choice="var", axes=1, fill="pink", color="black", top=5)
p2 <- fviz_contrib(pca, choice="var", axes=2, fill="skyblue", color="black", top=5)
p3 <- fviz_contrib(pca, choice="var", axes=3, fill="skyblue", color="black", top=5)
p4 <- fviz_contrib(pca, choice="var", axes=4, fill="skyblue", color="black", top=5)
p5 <- fviz_contrib(pca, choice="var", axes=5, fill="skyblue", color="black", top=5)
grid.arrange(p1,p2,p3,p4,p5,ncol=5)

Here we are sourcing the function directly from package without loading it into the kernel.(i could not install the package)


In [ ]:
source('https://raw.githubusercontent.com/vqv/ggbiplot/master/R/ggbiplot.r')

source('https://raw.githubusercontent.com/vqv/ggbiplot/master/R/ggscreeplot.r')

Here is a visual representation if a PCA.
The first chart explains the cumulative persent contibution of PCA.
Second is the PCA plot itself where the componets are clearly separable.

In [ ]:
options(repr.plot.width=16, repr.plot.height = 10)

g <- ggbiplot(pca,
              obs.scale = 1,
              var.scale = 1,
              groups = train$diagnosis,
              ellipse = TRUE,
              circle = TRUE,
              ellipse.prob = 0.68)
g <- g + scale_color_discrete(name = '')
g <- g + theme(legend.direction = 'horizontal',
               legend.position = 'top')
pca.ex=ggscreeplot(pca)+ geom_vline(xintercept = 11:15, colour="purple", linetype = "longdash")
grid.arrange(pca.ex,g,ncol=2)

Here we re spliting the dataset to prepare for modeling.

In [ ]:
train.pca <- predict(pca, train)
train.pca <- data.frame(train.pca, train[1])
test.pca <- predict(pca, test)
test.pca <- data.frame(test.pca, test[1])


#2.0
## 🍒 Model & Validate The Data

MODELS


Simple machine learning tools are used from classification category.

#GLM

The first algo is a The Holy Logistic Regression.
Predicting diagnosis using PC1 and PC2, model accuracy is 95% Test and 94%Train ;Thank You , Glm!


In [ ]:
library(nnet)

GLM = multinom(diagnosis~PC1+PC2, data = train.pca)
glm.prediction.train = predict(GLM, train.pca)


comatrix.glm.train = confusionMatrix(glm.prediction.train, train.pca$diagnosis)
comatrix.glm.train


In [ ]:
glm.prediction.test= predict(GLM, test.pca)



comatrix.glm.test = confusionMatrix(glm.prediction.test, test.pca$diagnosis)
comatrix.glm.test

This is used for cross validation in some models

In [ ]:
library(caret)
control <- trainControl(method='repeatedcv', 
                        number=13, 
                        repeats=3)


#KNN

KNN:K-Nearest Neigbour 
KNN is a standart classification function that uses euclidian distance  to calculate desision boundaies depending on number of observations K.
Usually , the knn curve has a low accuracy for low number of neighbours , peaking at 5-10 .

In [ ]:
knn10<- train(diagnosis ~ PC1+PC2, data = train.pca, method = "knn", trControl = control,tuneLength = 4)
knn10.1=plot(knn10)
knn100<- train(diagnosis ~ PC1+PC2, data = train.pca, method = "knn", trControl = control,tuneLength = 20)
knn100.1=plot(knn100)
knn1000<- train(diagnosis ~ PC1+PC2, data = train.pca, method = "knn", trControl = control,tuneLength = 60)
knn1000.1=plot(knn1000)
grid.arrange(knn10.1,knn100.1,knn1000.1,ncol=3)
options(repr.plot.width=30, repr.plot.height = 10)

#2

In [ ]:
knn.prediction.train= predict(knn10, train.pca)
comatrix.knn.train = confusionMatrix(knn.prediction.train, train.pca$diagnosis, positive = "M")
comatrix.knn.train

In [ ]:
glm.prediction.test= predict(knn10, test.pca)
comatrix.knn.test = confusionMatrix(glm.prediction.test, test.pca$diagnosis, positive = "M")
comatrix.knn.test

#SVM

In [ ]:
options(repr.plot.width=10, repr.plot.height = 10)
library(e1071)

SVM <- svm(diagnosis ~ PC1+PC2, data = train.pca, kernel = "linear", cost = 13)
svm.1=plot(SVM,train.pca,PC1~PC2)



In [ ]:
svm.prediction.train<- predict(SVM, train.pca)
comatrix.svm.train <- confusionMatrix(svm.prediction.train, train.pca$diagnosis, positive = "M")
comatrix.svm.train

In [ ]:
svm.prediction.test<- predict(SVM, test.pca)
comatrix.svm.test <- confusionMatrix(svm.prediction.test, test.pca$diagnosis, positive = "M")
comatrix.svm.test

In [ ]:
SVM.r <- svm(diagnosis ~ PC1+PC2, data = train.pca, kernel = "radial", cost = 40)
svm.2=plot(SVM.r,train.pca,PC1~PC2)

In [ ]:
svm.prediction.train<- predict(SVM.r, train.pca)
comatrix.svm.train <- confusionMatrix(svm.prediction.train, train.pca$diagnosis, positive = "M")
comatrix.svm.train

In [ ]:
svm.prediction.test<- predict(SVM.r, test.pca)
comatrix.svm.test <- confusionMatrix(svm.prediction.test, test.pca$diagnosis, positive = "M")
comatrix.svm.test

#C5

In [ ]:
library(C50)
C50 <- C5.0(train.pca,train.pca$diagnosis)

c50.prediction.train <- predict(C50, train.pca)
c50.comatrix.train <- confusionMatrix(c50.prediction.train, train.pca$diagnosis)
c50.comatrix.train

c50.prediction.test <- predict(C50, test.pca)
c50.comatrix.test<- confusionMatrix(c50.prediction.test, test.pca$diagnosis)
c50.comatrix.test
plot(C50)

#csC5

In [ ]:
options(repr.plot.width=16, repr.plot.height = 10)
mod2 <- train(diagnosis ~ PC1+PC2, data = train.pca, 
              method = "C5.0Cost",
              tuneGrid = expand.grid(model = "tree", winnow = FALSE,
                                     trials = c(1:10, (1:5)*10),
                                     cost = 1:10),
              trControl = control)

plot(mod2)

In [ ]:
c50c.predict.train <- predict(mod2, train.pca)
c50c.comatrix.train <- confusionMatrix(c50c.predict.train, train.pca$diagnosis)
c50c.comatrix.train

In [ ]:
c50c.predict.test <- predict(mod2, test.pca)
c50c.comatrix.test  <- confusionMatrix(c50c.predict.test, test.pca$diagnosis)
c50c.comatrix.test

#3.0
## 🥒 Conclude The Data

In [ ]:
options(repr.plot.width=26, repr.plot.height = 15)
col <- c("#06908F", "#dbb68f")
par(mfrow=c(2,6))
fourfoldplot(comatrix.glm.train$table, color = col, conf.level = 0, margin = 1, main=paste("Logistic.train (",round(comatrix.glm.train$overall[1]*100),"%)",sep=""))
fourfoldplot(comatrix.glm.test$table, color = col, conf.level = 0, margin = 1, main=paste("Logistic.test (",round(comatrix.glm.test$overall[1]*100),"%)",sep=""))

fourfoldplot(c50.comatrix.train$table, color = col, conf.level = 0, margin = 1, main=paste("C5.0.train (",round(c50.comatrix.train$overall[1]*100),"%)",sep=""))
fourfoldplot(c50.comatrix.test$table, color = col, conf.level = 0, margin = 1, main=paste("C5.0.test (",round(c50.comatrix.test$overall[1]*100),"%)",sep=""))

fourfoldplot(c50c.comatrix.train$table, color = col, conf.level = 0, margin = 1, main=paste("Cost Sensitive C5.0.train (",round(c50c.comatrix.train$overall[1]*100),"%)",sep=""))
fourfoldplot(c50c.comatrix.test$table, color = col, conf.level = 0, margin = 1, main=paste("Cost Sensitive C5.0.test (",round(c50c.comatrix.test$overall[1]*100),"%)",sep=""))

fourfoldplot(comatrix.svm.train$table, color = col, conf.level = 0, margin = 1, main=paste("SVM.train (",round(comatrix.svm.train$overall[1]*100),"%)",sep=""))
fourfoldplot(comatrix.svm.test$table, color = col, conf.level = 0, margin = 1, main=paste("SVM.test (",round(comatrix.svm.test$overall[1]*100),"%)",sep=""))

fourfoldplot(comatrix.knn.train$table, color = col, conf.level = 0, margin = 1, main=paste("KNN.train(",round(comatrix.knn.train$overall[1]*100),"%)",sep=""))
fourfoldplot(comatrix.knn.test$table, color = col, conf.level = 0, margin = 1, main=paste("KNN.test(",round(comatrix.knn.test$overall[1]*100),"%)",sep=""))
